# Agent-Based Clinical Data Extraction Prototype

**Goal:** Build an agent using OpenAI Agents SDK to extract structured clinical data from SOAP notes and enrich with ICD-10-CM and RxNorm codes.

**Architecture:**
1. Define function tools for extraction and code lookup
2. Create agent with structured output
3. Test on SOAP notes
4. Once working, integrate into backend API


## Section 1: Imports and Configuration


In [63]:
import os
import json
import asyncio
from typing import List, Optional, Dict
from pathlib import Path
from datetime import datetime, timezone

# OpenAI Agents SDK
from agents import Agent, Runner, function_tool

# HTTP client for external APIs
import httpx

# Pydantic for structured output
from pydantic import BaseModel, Field

# OpenAI client (for direct LLM calls if needed)
from openai import OpenAI

print("✅ All imports successful")


✅ All imports successful


Load Environment variables

In [64]:
from dotenv import load_dotenv
load_dotenv("../.env")

# Verify OpenAI API key is set
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in environment")

print("✅ OpenAI API key loaded")
print(f"   Key ends with: ...{OPENAI_API_KEY[-5:]}")


✅ OpenAI API key loaded
   Key ends with: ...EcNwA


## Section 2: Test External API Access

Before building tools, let's verify we can access the NLM APIs.


In [75]:
# Test NLM Clinical Tables API (ICD-10-CM)
async def test_icd_api(search_term: str):
    async with httpx.AsyncClient() as client:
        response = await client.get(
            "https://clinicaltables.nlm.nih.gov/api/icd10cm/v3/search",
            params={"sf": "code,name", "terms": search_term, "maxList": 3}
        )
        data = response.json()
        print("ICD-10-CM API Response:")
        print(json.dumps(data, indent=2))
        return data

# Pass in complete or partial search terms to test
await test_icd_api("leukem")
print("✅ ICD-10-CM API accessible")


ICD-10-CM API Response:
[
  96,
  [
    "C90.10",
    "C90.11",
    "C90.12"
  ],
  null,
  [
    [
      "C90.10",
      "Plasma cell leukemia not having achieved remission"
    ],
    [
      "C90.11",
      "Plasma cell leukemia in remission"
    ],
    [
      "C90.12",
      "Plasma cell leukemia in relapse"
    ]
  ]
]
✅ ICD-10-CM API accessible


## Section 3: Define Pydantic Schemas

Define the structured output format for the agent.


In [72]:
class PatientInfo(BaseModel):
    """Patient demographic information."""
    age: Optional[str] = None
    gender: Optional[str] = None
    
class VitalSigns(BaseModel):
    """Patient vital signs extracted from note."""
    temperature: Optional[str] = None
    blood_pressure: Optional[str] = None
    heart_rate: Optional[str] = None
    respiratory_rate: Optional[str] = None
    oxygen_saturation: Optional[str] = None
    weight: Optional[str] = None
    height: Optional[str] = None
    bmi: Optional[str] = None


class DiagnosisCode(BaseModel):
    """Diagnosis with ICD-10-CM code enrichment."""
    text: str = Field(description="Original diagnosis text from note")
    icd10_code: Optional[str] = Field(None, description="ICD-10-CM code")
    icd10_description: Optional[str] = Field(None, description="ICD-10-CM code description")
    confidence: Optional[str] = Field(None, description="Confidence level: exact, high, low, or none")


class MedicationCode(BaseModel):
    """Medication with RxNorm code enrichment."""
    text: str = Field(description="Original medication text from note")
    rxnorm_code: Optional[str] = Field(None, description="RxNorm RxCUI")
    rxnorm_name: Optional[str] = Field(None, description="RxNorm normalized name")
    confidence: Optional[str] = Field(None, description="Confidence level: exact, approximate, or none")


class StructuredClinicalData(BaseModel):
    """Final structured output from agent."""
    patient_info: Optional[PatientInfo] = Field(None, description="Patient demographics if available")
    diagnoses: List[DiagnosisCode] = Field(default_factory=list, description="Conditions and diagnoses with ICD codes")
    medications: List[MedicationCode] = Field(default_factory=list, description="Medications with RxNorm codes")
    vital_signs: Optional[VitalSigns] = Field(None, description="Vital signs")
    lab_results: List[str] = Field(default_factory=list, description="Laboratory test results")
    plan_actions: List[str] = Field(default_factory=list, description="Treatment plan and follow-up actions")

print("✅ Pydantic schemas defined")


✅ Pydantic schemas defined


## Section 4: Define Function Tools

Create the three function tools the agent will use.


In [73]:
# Tool 1: Extract clinical entities from text
# Define the raw function first (for testing)
def extract_clinical_entities_func(note_text: str) -> dict:
    """
    Extract clinical entities from a medical note.
    
    Returns a dictionary with:
    - diagnoses: list of diagnosis/condition strings
    - medications: list of medication strings
    - vital_signs: dict of vital sign measurements
    - lab_results: list of lab test result strings
    - plan_actions: list of treatment plan items
    - patient_info: dict of patient demographics if available
    """
    # This tool will use LLM to extract entities
    # The agent will handle the LLM call
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    system_prompt = """
        You are a medical data extraction specialist. Extract clinical entities from medical notes.

        Extract and return JSON with these fields:
        - diagnoses: array of diagnosis/condition strings (e.g., ["Type 2 Diabetes Mellitus", "Hypertension"])
        - medications: array of medication strings with dosages if present (e.g., ["Metformin 500mg", "Lisinopril 10mg"])
        - vital_signs: object with keys like temperature, blood_pressure, heart_rate, respiratory_rate, oxygen_saturation, weight, height, bmi (all optional)
        - lab_results: array of lab result strings (e.g., ["HbA1c: 7.2%", "eGFR: 85 mL/min"])
        - plan_actions: array of treatment plan items (e.g., ["Follow-up in 3 months", "Continue current medications"])
        - patient_info: object with demographics like age, gender if mentioned

        Extract exactly as written in the note. Do not normalize or interpret.
    """
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {"role": "user", "content": note_text}
        ],
        response_format={"type": "json_object"}
    )
    
    return json.loads(response.choices[0].message.content)

# Create the tool wrapper for the agent
extract_clinical_entities = function_tool(extract_clinical_entities_func)

print("✅ Tool 1: extract_clinical_entities defined")


✅ Tool 1: extract_clinical_entities defined


In [74]:
# Tool 2: Lookup ICD-10-CM code
# Define the raw function first (for testing)
async def lookup_icd10_code_func(condition: str) -> dict:
    """
    Look up ICD-10-CM code for a medical condition using NLM Clinical Tables API.
    
    Args:
        condition: The condition or diagnosis text (e.g., "type 2 diabetes")
    
    Returns:
        dict with code, name (description), and confidence level
    """
    try:
        async with httpx.AsyncClient(timeout=10.0) as client:
            response = await client.get(
                "https://clinicaltables.nlm.nih.gov/api/icd10cm/v3/search",
                params={
                    "sf": "code,name",
                    "terms": condition,
                    "maxList": 1
                }
            )
            response.raise_for_status()
            data = response.json()
            
            # Response format: [count, [codes], null, [[code, name]]]
            count = data[0]
            if count > 0 and len(data) > 3 and data[3]:
                code_info = data[3][0]  # First result
                return {
                    "code": code_info[0],
                    "description": code_info[1],
                    "confidence": "high" if count == 1 else "exact"
                }
            else:
                return {
                    "code": None,
                    "description": None,
                    "confidence": "none"
                }
    except Exception as e:
        print(f"Error looking up ICD code for '{condition}': {e}")
        return {
            "code": None,
            "description": None,
            "confidence": "none",
            "error": str(e)
        }

# Create the tool wrapper for the agent
lookup_icd10_code = function_tool(lookup_icd10_code_func)

print("✅ Tool 2: lookup_icd10_code defined")


✅ Tool 2: lookup_icd10_code defined


In [76]:
# Tool 3: Lookup RxNorm code
# Define the raw function first (for testing)
async def lookup_rxnorm_code_func(medication: str) -> dict:
    """
    Look up RxNorm code (RxCUI) for a medication using NLM RxNav API.
    
    Args:
        medication: The medication text (e.g., "metformin 500mg")
    
    Returns:
        dict with rxcui, name, and confidence level
    """
    try:
        async with httpx.AsyncClient(timeout=10.0) as client:
            # Try exact match first
            response = await client.get(
                "https://rxnav.nlm.nih.gov/REST/rxcui.json",
                params={"name": medication}
            )
            response.raise_for_status()
            data = response.json()
            
            if "idGroup" in data and "rxnormId" in data["idGroup"]:
                rxcui = data["idGroup"]["rxnormId"][0]
                
                # Get the name for this RxCUI
                name_response = await client.get(
                    f"https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/property.json",
                    params={"propName": "RxNorm Name"}
                )
                name_data = name_response.json()
                
                rxnorm_name = medication  # Default to input
                if "propConceptGroup" in name_data:
                    props = name_data["propConceptGroup"].get("propConcept", [])
                    if props:
                        rxnorm_name = props[0].get("propValue", medication)
                
                return {
                    "rxcui": rxcui,
                    "name": rxnorm_name,
                    "confidence": "exact"
                }
            
            # Try approximate match if exact fails
            approx_response = await client.get(
                "https://rxnav.nlm.nih.gov/REST/approximateTerm.json",
                params={"term": medication, "maxEntries": 1}
            )
            approx_data = approx_response.json()
            
            if "approximateGroup" in approx_data:
                candidates = approx_data["approximateGroup"].get("candidate", [])
                if candidates:
                    best = candidates[0]
                    return {
                        "rxcui": best.get("rxcui"),
                        "name": best.get("name", medication),
                        "confidence": "approximate"
                    }
            
            return {
                "rxcui": None,
                "name": None,
                "confidence": "none"
            }
            
    except Exception as e:
        print(f"Error looking up RxNorm code for '{medication}': {e}")
        return {
            "rxcui": None,
            "name": None,
            "confidence": "none",
            "error": str(e)
        }

# Create the tool wrapper for the agent
lookup_rxnorm_code = function_tool(lookup_rxnorm_code_func)

print("✅ Tool 3: lookup_rxnorm_code defined")


✅ Tool 3: lookup_rxnorm_code defined


## Section 5: Test Tools Independently

Before creating the agent, test each tool individually.


In [77]:
# Test Tool 1: Extract entities from sample text
sample_note = """
Subjective: 45-year-old male with Type 2 Diabetes Mellitus presents for follow-up. 
Reports good compliance with Metformin 500mg twice daily. Blood sugars have been stable.

Objective: BP 130/85, HR 72, Temp 98.6°F. Weight 180 lbs.

Assessment: Type 2 Diabetes Mellitus, well-controlled. Hypertension, stable.

Plan: Continue Metformin. Start Lisinopril 10mg daily for BP control. 
Follow-up in 3 months with lab work (HbA1c, lipid panel).
"""

print("Testing extract_clinical_entities tool...")
result = extract_clinical_entities_func(sample_note)
print("\nExtracted entities:")
print(json.dumps(result, indent=2))

# Test Tool 2: ICD lookup
print("\n" + "="*80)
print("Testing ICD-10-CM lookup...")
print("="*80)
test_conditions = ["type 2 diabetes", "hypertension", "acute otitis media"]

for condition in test_conditions:
    result = await lookup_icd10_code_func(condition)
    print(f"\nCondition: {condition}")
    print(f"  Code: {result.get('code')}")
    print(f"  Description: {result.get('description')}")
    print(f"  Confidence: {result.get('confidence')}")


Testing extract_clinical_entities tool...

Extracted entities:
{
  "diagnoses": [
    "Type 2 Diabetes Mellitus",
    "Hypertension"
  ],
  "medications": [
    "Metformin 500mg twice daily",
    "Lisinopril 10mg daily"
  ],
  "vital_signs": {
    "blood_pressure": "130/85",
    "heart_rate": 72,
    "temperature": "98.6\u00b0F",
    "weight": "180 lbs"
  },
  "lab_results": [
    "HbA1c",
    "lipid panel"
  ],
  "plan_actions": [
    "Continue Metformin",
    "Start Lisinopril 10mg daily for BP control",
    "Follow-up in 3 months with lab work"
  ],
  "patient_info": {
    "age": 45,
    "gender": "male"
  }
}

Testing ICD-10-CM lookup...

Condition: type 2 diabetes
  Code: E11.65
  Description: Type 2 diabetes mellitus with hyperglycemia
  Confidence: exact

Condition: hypertension
  Code: I15.0
  Description: Renovascular hypertension
  Confidence: exact

Condition: acute otitis media
  Code: H65.03
  Description: Acute serous otitis media, bilateral
  Confidence: exact


In [78]:
# Test NLM RxNav API (RxNorm)
async def test_rxnorm_api():
    async with httpx.AsyncClient() as client:
        # Test exact match
        response = await client.get(
            "https://rxnav.nlm.nih.gov/REST/rxcui.json",
            params={"name": "metformin"}
        )
        data = response.json()
        print("RxNorm API Response (exact match):")
        print(json.dumps(data, indent=2))
        return data

await test_rxnorm_api()
print("✅ RxNorm API accessible")


RxNorm API Response (exact match):
{
  "idGroup": {
    "rxnormId": [
      "6809"
    ]
  }
}
✅ RxNorm API accessible


In [39]:
# Test Tool 3: RxNorm lookup
print("\n" + "="*80)
print("Testing RxNorm lookup...")
print("="*80)
test_medications = ["metformin", "lisinopril 10mg", "ibuprofen 200mg"]

for medication in test_medications:
    result = await lookup_rxnorm_code_func(medication)
    print(f"\nMedication: {medication}")
    print(f"  RxCUI: {result.get('rxcui')}")
    print(f"  Name: {result.get('name')}")
    print(f"  Confidence: {result.get('confidence')}")

print("\n✅ All tools tested successfully!")



Testing RxNorm lookup...

Medication: metformin
  RxCUI: 6809
  Name: metformin
  Confidence: exact

Medication: lisinopril 10mg
  RxCUI: 316151
  Name: lisinopril 10 MG
  Confidence: approximate

Medication: ibuprofen 200mg
  RxCUI: 316074
  Name: ibuprofen 200 MG
  Confidence: approximate

✅ All tools tested successfully!


### Create the Agent

In [79]:
# Create the extraction agent
extraction_agent = Agent(
    name="Clinical Data Extraction Agent",
    instructions="""
        You are a medical data extraction specialist. Your job is to extract structured clinical data from medical notes and enrich it with standardized medical codes.

        **Process:**
        1. First, use the extract_clinical_entities tool to extract raw clinical data from the note
        2. For each diagnosis/condition extracted, use lookup_icd10_code to get the ICD-10-CM code
        3. For each medication extracted, use lookup_rxnorm_code to get the RxNorm code
        4. Compile all the enriched data into the structured output format

        **Important guidelines:**
        - Be thorough - extract all diagnoses, medications, vital signs, labs, and plan items
        - Call the lookup tools for EVERY diagnosis and medication (don't skip any)
        - If a code lookup fails or returns no results, still include the text with null codes
        - Preserve the original text from the note in the 'text' field
        - Handle missing or incomplete data gracefully
        - The final output must match the StructuredClinicalData schema exactly
    """,
    tools=[extract_clinical_entities, lookup_icd10_code, lookup_rxnorm_code],
    output_type=StructuredClinicalData,
)

print("✅ Agent created with 3 tools and structured output type")
print(f"   Agent name: {extraction_agent.name}")
print(f"   Number of tools: {len(extraction_agent.tools)}")
print(f"   Output type: {extraction_agent.output_type.__name__}")


✅ Agent created with 3 tools and structured output type
   Agent name: Clinical Data Extraction Agent
   Number of tools: 3
   Output type: StructuredClinicalData



#### Load the Test SOAP note

In [80]:
# Load a SOAP note for testing
soap_path = Path("../../med_docs/soap/soap_01.txt")

if soap_path.exists():
    with open(soap_path, "r") as f:
        soap_01 = f.read()
    print("✅ Loaded SOAP note 01")
    print(f"\nNote preview (first 300 chars):\n{soap_01[:300]}...")
else:
    print("❌ SOAP note file not found")
    soap_01 = None


✅ Loaded SOAP note 01

Note preview (first 300 chars):
SOAP Note - Encounter Date: 2023-10-26
Patient: patient--001

S: Pt presents today for annual physical check-up. No chief complaints. Reports generally good health, denies chest pain, SOB, HA, dizziness. Family hx of elevated cholesterol (dad), no significant personal PMH issues reported. States rou...


In [81]:
# Run the agent on the SOAP note
if soap_01:
    print("🤖 Running agent on SOAP note 01...")
    print("   This may take 30-60 seconds as the agent calls multiple tools...\n")
    
    result = await Runner.run(
        extraction_agent,
        input=f"Extract and enrich clinical data from this medical note:\n\n{soap_01}"
    )
    
    print("\n" + "="*80)
    print("AGENT EXECUTION COMPLETE")
    print("="*80)
    
    # The final_output should be a StructuredClinicalData instance
    structured_data = result.final_output
    
    print("\n📊 Structured Clinical Data (JSON):\n")
    print(json.dumps(structured_data.model_dump(), indent=2))


🤖 Running agent on SOAP note 01...
   This may take 30-60 seconds as the agent calls multiple tools...


AGENT EXECUTION COMPLETE

📊 Structured Clinical Data (JSON):

{
  "patient_info": {
    "age": null,
    "gender": null
  },
  "diagnoses": [
    {
      "text": "Possible overweight",
      "icd10_code": "E66.3",
      "icd10_description": "Overweight",
      "confidence": "high"
    }
  ],
  "medications": [
    {
      "text": "Routine annual influenza vaccine",
      "rxnorm_code": "5806",
      "rxnorm_name": "influenza vaccine",
      "confidence": "approximate"
    }
  ],
  "vital_signs": {
    "temperature": "98.2\u00c2\u00b0F oral",
    "blood_pressure": "128/82 mmHg",
    "heart_rate": "72 bpm",
    "respiratory_rate": "16 breaths/min",
    "oxygen_saturation": null,
    "weight": "192 lbs",
    "height": "5'10\"",
    "bmi": "27.5"
  },
  "lab_results": [
    "CBC",
    "CMP",
    "Lipid panel"
  ],
  "plan_actions": [
    "Advised pt on healthier diet",
    "Increasing w

In [82]:
# Pretty print the results by category
if soap_01 and structured_data:
    print("\n" + "="*80)
    print("FORMATTED OUTPUT")
    print("="*80)
    
    print("\n🏥 DIAGNOSES (with ICD-10-CM codes):")
    if structured_data.diagnoses:
        for dx in structured_data.diagnoses:
            print(f"  • {dx.text}")
            if dx.icd10_code:
                print(f"    └─ ICD-10: {dx.icd10_code} - {dx.icd10_description}")
                print(f"       Confidence: {dx.confidence}")
            else:
                print(f"    └─ No ICD code found")
    else:
        print("  (None found)")
    
    print("\n💊 MEDICATIONS (with RxNorm codes):")
    if structured_data.medications:
        for med in structured_data.medications:
            print(f"  • {med.text}")
            if med.rxnorm_code:
                print(f"    └─ RxNorm: {med.rxnorm_code} - {med.rxnorm_name}")
                print(f"       Confidence: {med.confidence}")
            else:
                print(f"    └─ No RxNorm code found")
    else:
        print("  (None found)")
    
    print("\n📈 VITAL SIGNS:")
    if structured_data.vital_signs:
        vital_dict = structured_data.vital_signs.model_dump()
        has_vitals = any(v is not None for v in vital_dict.values())
        if has_vitals:
            for key, value in vital_dict.items():
                if value:
                    print(f"  • {key.replace('_', ' ').title()}: {value}")
        else:
            print("  (None found)")
    else:
        print("  (None found)")
    
    print("\n🔬 LAB RESULTS:")
    if structured_data.lab_results:
        for lab in structured_data.lab_results:
            print(f"  • {lab}")
    else:
        print("  (None found)")
    
    print("\n📋 PLAN ACTIONS:")
    if structured_data.plan_actions:
        for action in structured_data.plan_actions:
            print(f"  • {action}")
    else:
        print("  (None found)")
    
    print("\n👤 PATIENT INFO:")
    if structured_data.patient_info:
        patient_dict = structured_data.patient_info.model_dump()
        has_info = any(v is not None for v in patient_dict.values())
        if has_info:
            for key, value in patient_dict.items():
                if value:
                    print(f"  • {key.title()}: {value}")
        else:
            print("  (None found)")
    else:
        print("  (None found)")
    
    # Summary statistics
    print("\n" + "="*80)
    print("SUMMARY STATISTICS")
    print("="*80)
    icd_count = sum(1 for dx in structured_data.diagnoses if dx.icd10_code)
    rxnorm_count = sum(1 for med in structured_data.medications if med.rxnorm_code)
    
    print(f"✅ Extracted {len(structured_data.diagnoses)} diagnoses ({icd_count} with ICD-10-CM codes)")
    print(f"✅ Extracted {len(structured_data.medications)} medications ({rxnorm_count} with RxNorm codes)")
    print(f"✅ Extracted {len(structured_data.lab_results)} lab results")
    print(f"✅ Extracted {len(structured_data.plan_actions)} plan actions")



FORMATTED OUTPUT

🏥 DIAGNOSES (with ICD-10-CM codes):
  • Possible overweight
    └─ ICD-10: E66.3 - Overweight
       Confidence: high

💊 MEDICATIONS (with RxNorm codes):
  • Routine annual influenza vaccine
    └─ RxNorm: 5806 - influenza vaccine
       Confidence: approximate

📈 VITAL SIGNS:
  • Temperature: 98.2Â°F oral
  • Blood Pressure: 128/82 mmHg
  • Heart Rate: 72 bpm
  • Respiratory Rate: 16 breaths/min
  • Weight: 192 lbs
  • Height: 5'10"
  • Bmi: 27.5

🔬 LAB RESULTS:
  • CBC
  • CMP
  • Lipid panel

📋 PLAN ACTIONS:
  • Advised pt on healthier diet
  • Increasing weekly exercise frequency to at least 3-4 times/week
  • Scheduled follow-up visit to review lab results and cholesterol levels in approx. 5 months

👤 PATIENT INFO:
  (None found)

SUMMARY STATISTICS
✅ Extracted 1 diagnoses (1 with ICD-10-CM codes)
✅ Extracted 1 medications (1 with RxNorm codes)
✅ Extracted 3 lab results
✅ Extracted 3 plan actions


In [83]:
# Test on multiple SOAP notes
soap_files = ["soap_01.txt", "soap_02.txt", "soap_03.txt"]
results = {}

for soap_file in soap_files:
    soap_path = Path(f"../../med_docs/soap/{soap_file}")
    
    if not soap_path.exists():
        print(f"⚠️  Skipping {soap_file} - file not found")
        continue
    
    with open(soap_path, "r") as f:
        soap_text = f.read()
    
    print(f"\n{'='*80}")
    print(f"Processing: {soap_file}")
    print(f"{'='*80}")
    
    result = await Runner.run(
        extraction_agent,
        input=f"Extract and enrich clinical data from this medical note:\n\n{soap_text}"
    )
    
    results[soap_file] = result.final_output
    
    # Summary stats
    data = result.final_output
    print(f"\n✅ Extracted:")
    print(f"   • {len(data.diagnoses)} diagnoses")
    print(f"   • {len(data.medications)} medications")
    print(f"   • {len(data.lab_results)} lab results")
    print(f"   • {len(data.plan_actions)} plan actions")
    
    # Check code enrichment
    icd_count = sum(1 for dx in data.diagnoses if dx.icd10_code)
    rxnorm_count = sum(1 for med in data.medications if med.rxnorm_code)
    print(f"\n✅ Code enrichment:")
    print(f"   • {icd_count}/{len(data.diagnoses)} diagnoses have ICD-10-CM codes")
    print(f"   • {rxnorm_count}/{len(data.medications)} medications have RxNorm codes")

print(f"\n{'='*80}")
print("TESTING COMPLETE")
print(f"{'='*80}")
print(f"✅ Processed {len(results)} SOAP notes successfully")



Processing: soap_01.txt

✅ Extracted:
   • 2 diagnoses
   • 1 medications
   • 3 lab results
   • 3 plan actions

✅ Code enrichment:
   • 0/2 diagnoses have ICD-10-CM codes
   • 1/1 medications have RxNorm codes

Processing: soap_02.txt

✅ Extracted:
   • 3 diagnoses
   • 1 medications
   • 3 lab results
   • 5 plan actions

✅ Code enrichment:
   • 1/3 diagnoses have ICD-10-CM codes
   • 1/1 medications have RxNorm codes

Processing: soap_03.txt

✅ Extracted:
   • 0 diagnoses
   • 0 medications
   • 0 lab results
   • 4 plan actions

✅ Code enrichment:
   • 0/0 diagnoses have ICD-10-CM codes
   • 0/0 medications have RxNorm codes

TESTING COMPLETE
✅ Processed 3 SOAP notes successfully


## Section 6: FHIR Conversion

Now that we have structured clinical data with ICD-10-CM and RxNorm codes, let's convert it to FHIR R4 resources.

**Goal:** Map our structured data to FHIR resources:
- Patient (demographics)
- Condition (diagnoses with ICD-10-CM codes)
- MedicationRequest (medications with RxNorm codes)
- Observation (vital signs with LOINC codes, lab results)


In [84]:
# FHIR resource imports
from datetime import datetime
from fhir.resources.patient import Patient
from fhir.resources.condition import Condition
from fhir.resources.medicationrequest import MedicationRequest
from fhir.resources.observation import Observation
from fhir.resources.bundle import Bundle

print("✅ FHIR resources library imported successfully")
print(f"   Available FHIR resources: Patient, Condition, MedicationRequest, Observation, Bundle")


✅ FHIR resources library imported successfully
   Available FHIR resources: Patient, Condition, MedicationRequest, Observation, Bundle


In [85]:
# Verify we have structured data from agent extraction to work with
print("=" * 80)
print("Available data from agent extraction:")
print("=" * 80)

if 'structured_data' in locals():
    print(f"✅ structured_data available")
    print(f"   • Patient info: {structured_data.patient_info is not None}")
    print(f"   • Diagnoses: {len(structured_data.diagnoses)}")
    print(f"   • Medications: {len(structured_data.medications)}")
    print(f"   • Vital signs: {structured_data.vital_signs is not None}")
    print(f"   • Lab results: {len(structured_data.lab_results)}")
    print(f"   • Plan actions: {len(structured_data.plan_actions)}")
    
    # Show a quick example of what we have
    if structured_data.diagnoses:
        print(f"\n📋 Example diagnosis to convert:")
        dx = structured_data.diagnoses[0]
        print(f"   Text: {dx.text}")
        print(f"   ICD-10: {dx.icd10_code} - {dx.icd10_description}")
else:
    print("⚠️  No structured_data found. Run the agent extraction cells above first.")
    print("   (Cell 36: Run agent on SOAP note)")


Available data from agent extraction:
✅ structured_data available
   • Patient info: True
   • Diagnoses: 1
   • Medications: 1
   • Vital signs: True
   • Lab results: 3
   • Plan actions: 3

📋 Example diagnosis to convert:
   Text: Possible overweight
   ICD-10: E66.3 - Overweight


#### Function to create Patient FHIR resource

In [86]:
def map_patient_info_to_fhir(patient_info, patient_id: str = "unknown"):
    """
    Map PatientInfo to FHIR Patient resource.
    
    Returns None if no patient info available.
    """
    if not patient_info or (not patient_info.age and not patient_info.gender):
        return None
    
    patient_data = {
        "resourceType": "Patient",
        "id": patient_id,
        "identifier": [{
            "system": "urn:oid:df-healthbench",
            "value": patient_id
        }]
    }
    
    # Map gender
    if patient_info.gender:
        gender_map = {"male": "male", "female": "female", "m": "male", "f": "female"}
        patient_data["gender"] = gender_map.get(patient_info.gender.lower(), "unknown")
    
    # Derive birthDate from age (approximate)
    if patient_info.age:
        try:
            age_int = int(patient_info.age)
            birth_year = datetime.now().year - age_int
            patient_data["birthDate"] = f"{birth_year}-01-01"
        except ValueError:
            print(f"Warning: Could not parse age: {patient_info.age}")
    
    return Patient(**patient_data)

# Test with our data
patient_resource = map_patient_info_to_fhir(structured_data.patient_info, "patient-soap-01")
if patient_resource:
    print("✅ Patient resource created:")
    print(json.dumps(patient_resource.model_dump(mode='json'), indent=2))
else:
    print("ℹ️  No patient info to convert")


ℹ️  No patient info to convert


#### Function to create Condition FHIR resource

In [87]:
def map_diagnosis_to_condition(diagnosis, patient_id: str):
    """
    Map DiagnosisCode to FHIR Condition resource.
    
    Uses ICD-10-CM code if available.
    """
    # Build coding array
    coding_list = []
    if diagnosis.icd10_code:
        coding_list.append({
            "system": "http://hl7.org/fhir/sid/icd-10-cm",
            "code": diagnosis.icd10_code,
            "display": diagnosis.icd10_description or diagnosis.text
        })
    
    condition_data = {
        "resourceType": "Condition",
        "clinicalStatus": {
            "coding": [{
                "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
                "code": "active"
            }]
        },
        "verificationStatus": {
            "coding": [{
                "system": "http://terminology.hl7.org/CodeSystem/condition-ver-status",
                "code": "confirmed"
            }]
        },
        "code": {
            "coding": coding_list if coding_list else None,
            "text": diagnosis.text
        },
        "subject": {"reference": f"Patient/{patient_id}"},
        "recordedDate": datetime.now(timezone.utc).isoformat()
    }
    
    return Condition(**condition_data)

# Test with first diagnosis
if structured_data.diagnoses:
    condition = map_diagnosis_to_condition(structured_data.diagnoses[0], "patient-soap-01")
    print("✅ Condition resource created:")
    print(json.dumps(condition.model_dump(mode='json'), indent=2))
else:
    print("ℹ️  No diagnoses to convert")


✅ Condition resource created:
{
  "resourceType": "Condition",
  "clinicalStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "active"
      }
    ]
  },
  "verificationStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/condition-ver-status",
        "code": "confirmed"
      }
    ]
  },
  "code": {
    "coding": [
      {
        "system": "http://hl7.org/fhir/sid/icd-10-cm",
        "code": "E66.3",
        "display": "Overweight"
      }
    ],
    "text": "Possible overweight"
  },
  "subject": {
    "reference": "Patient/patient-soap-01"
  },
  "recordedDate": "2025-11-30T18:09:53.219191Z"
}


#### Function to create MedicationRequest FHIR resource

In [88]:
def map_medication_to_request(medication, patient_id: str):
    """
    Map MedicationCode to FHIR MedicationRequest resource.
    
    Uses RxNorm code if available.
    """
    from fhir.resources.codeableconcept import CodeableConcept
    from fhir.resources.codeablereference import CodeableReference
    from fhir.resources.coding import Coding
    
    # Build coding array
    coding_list = []
    if medication.rxnorm_code:
        coding_list.append(
            Coding(
                system="http://www.nlm.nih.gov/research/umls/rxnorm",
                code=medication.rxnorm_code,
                display=medication.rxnorm_name or medication.text
            )
        )
    
    # Create CodeableConcept for medication
    medication_concept = CodeableConcept(
        coding=coding_list if coding_list else None,
        text=medication.text
    )
    
    # Wrap in CodeableReference (required by FHIR R4/R5)
    medication_ref = CodeableReference(
        concept=medication_concept
    )
    
    med_request_data = {
        "resourceType": "MedicationRequest",
        "status": "active",
        "intent": "order",
        "medication": medication_ref,
        "subject": {"reference": f"Patient/{patient_id}"},
        "authoredOn": datetime.now(timezone.utc).isoformat()
    }
    
    return MedicationRequest(**med_request_data)

# Test with first medication
if structured_data.medications:
    med_request = map_medication_to_request(structured_data.medications[0], "patient-soap-01")
    print("✅ MedicationRequest resource created:")
    print(json.dumps(med_request.model_dump(mode='json'), indent=2))
else:
    print("ℹ️  No medications to convert")


✅ MedicationRequest resource created:
{
  "resourceType": "MedicationRequest",
  "status": "active",
  "intent": "order",
  "medication": {
    "concept": {
      "coding": [
        {
          "system": "http://www.nlm.nih.gov/research/umls/rxnorm",
          "code": "5806",
          "display": "influenza vaccine"
        }
      ],
      "text": "Routine annual influenza vaccine"
    }
  },
  "subject": {
    "reference": "Patient/patient-soap-01"
  },
  "authoredOn": "2025-11-30T18:10:20.036384Z"
}


#### Function to create Observation FHIR resource

In [89]:
def map_vital_signs_to_observations(vital_signs, patient_id: str):
    """
    Map VitalSigns to FHIR Observation resources.
    
    Creates one Observation per vital with LOINC codes.
    """
    if not vital_signs:
        return []
    
    observations = []
    
    # LOINC code mapping for common vitals
    vital_mappings = {
        "blood_pressure": ("85354-9", "Blood pressure panel"),
        "heart_rate": ("8867-4", "Heart rate"),
        "respiratory_rate": ("9279-1", "Respiratory rate"),
        "temperature": ("8310-5", "Body temperature"),
        "oxygen_saturation": ("2708-6", "Oxygen saturation"),
        "weight": ("29463-7", "Body weight"),
        "height": ("8302-2", "Body height"),
        "bmi": ("39156-5", "Body mass index")
    }
    
    for field_name, (loinc_code, display) in vital_mappings.items():
        value = getattr(vital_signs, field_name, None)
        if value:
            obs_data = {
                "resourceType": "Observation",
                "status": "final",
                "category": [{
                    "coding": [{
                        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
                        "code": "vital-signs"
                    }]
                }],
                "code": {
                    "coding": [{
                        "system": "http://loinc.org",
                        "code": loinc_code,
                        "display": display
                    }],
                    "text": display
                },
                "subject": {"reference": f"Patient/{patient_id}"},
                "effectiveDateTime": datetime.now(timezone.utc).isoformat(),
                "valueString": str(value)  # Simplified
            }
            observations.append(Observation(**obs_data))
    
    return observations

# Test with vital signs
vital_observations = map_vital_signs_to_observations(
    structured_data.vital_signs, 
    "patient-soap-01"
)
print(f"✅ Created {len(vital_observations)} vital sign Observations")
if vital_observations:
    print("\nFirst vital observation:")
    print(json.dumps(vital_observations[0].model_dump(mode='json'), indent=2))


✅ Created 7 vital sign Observations

First vital observation:
{
  "resourceType": "Observation",
  "status": "final",
  "category": [
    {
      "coding": [
        {
          "system": "http://terminology.hl7.org/CodeSystem/observation-category",
          "code": "vital-signs"
        }
      ]
    }
  ],
  "code": {
    "coding": [
      {
        "system": "http://loinc.org",
        "code": "85354-9",
        "display": "Blood pressure panel"
      }
    ],
    "text": "Blood pressure panel"
  },
  "subject": {
    "reference": "Patient/patient-soap-01"
  },
  "effectiveDateTime": "2025-11-30T18:10:34.586584Z",
  "valueString": "128/82 mmHg"
}


In [90]:
def convert_to_fhir(structured_data, patient_id: str = "unknown"):
    """
    Convert structured clinical data to FHIR R4 resources.
    
    Returns dict with patient, conditions, medications, observations.
    """
    result = {
        "patient": None,
        "conditions": [],
        "medications": [],
        "observations": []
    }
    
    # 1. Convert patient
    if structured_data.patient_info:
        patient = map_patient_info_to_fhir(structured_data.patient_info, patient_id)
        if patient:
            result["patient"] = patient.model_dump(mode='json')
    
    # 2. Convert diagnoses to Conditions
    for diagnosis in structured_data.diagnoses:
        condition = map_diagnosis_to_condition(diagnosis, patient_id)
        result["conditions"].append(condition.model_dump(mode='json'))
    
    # 3. Convert medications to MedicationRequests
    for medication in structured_data.medications:
        med_request = map_medication_to_request(medication, patient_id)
        result["medications"].append(med_request.model_dump(mode='json'))
    
    # 4. Convert vital signs to Observations
    if structured_data.vital_signs:
        vital_obs = map_vital_signs_to_observations(structured_data.vital_signs, patient_id)
        result["observations"].extend([obs.model_dump(mode='json') for obs in vital_obs])
    
    # 5. Convert lab results to Observations (simplified text-only)
    for idx, lab_text in enumerate(structured_data.lab_results):
        obs_data = {
            "resourceType": "Observation",
            "status": "final",
            "category": [{
                "coding": [{
                    "system": "http://terminology.hl7.org/CodeSystem/observation-category",
                    "code": "laboratory"
                }]
            }],
            "code": {"text": "Laboratory result"},
            "subject": {"reference": f"Patient/{patient_id}"},
            "effectiveDateTime": datetime.now(timezone.utc).isoformat(),
            "valueString": lab_text
        }
        result["observations"].append(Observation(**obs_data).model_dump(mode='json'))
    
    return result

print("✅ Main conversion function created")


✅ Main conversion function created


In [91]:
# Convert the structured data we extracted earlier
print("=" * 80)
print("Converting structured data to FHIR resources...")
print("=" * 80)

fhir_resources = convert_to_fhir(structured_data, patient_id="patient-soap-01")

# Summary
print(f"\n✅ FHIR Conversion Complete!")
print(f"   • Patient: {1 if fhir_resources['patient'] else 0}")
print(f"   • Conditions: {len(fhir_resources['conditions'])}")
print(f"   • Medications: {len(fhir_resources['medications'])}")
print(f"   • Observations: {len(fhir_resources['observations'])}")

total_resources = (
    (1 if fhir_resources['patient'] else 0) +
    len(fhir_resources['conditions']) +
    len(fhir_resources['medications']) +
    len(fhir_resources['observations'])
)
print(f"   • Total resources: {total_resources}")


Converting structured data to FHIR resources...

✅ FHIR Conversion Complete!
   • Patient: 0
   • Conditions: 1
   • Medications: 1
   • Observations: 10
   • Total resources: 12


In [92]:
# Pretty print each resource type
print("\n" + "=" * 80)
print("FHIR RESOURCES (Pretty Print)")
print("=" * 80)

if fhir_resources['patient']:
    print("\n🏥 PATIENT:")
    print(json.dumps(fhir_resources['patient'], indent=2))

if fhir_resources['conditions']:
    print(f"\n💊 CONDITIONS ({len(fhir_resources['conditions'])}):")
    for idx, condition in enumerate(fhir_resources['conditions'], 1):
        print(f"\nCondition {idx}:")
        print(json.dumps(condition, indent=2))

if fhir_resources['medications']:
    print(f"\n💉 MEDICATIONS ({len(fhir_resources['medications'])}):")
    for idx, med in enumerate(fhir_resources['medications'], 1):
        print(f"\nMedication {idx}:")
        print(json.dumps(med, indent=2))

if fhir_resources['observations']:
    print(f"\n📊 OBSERVATIONS ({len(fhir_resources['observations'])}):")
    for idx, obs in enumerate(fhir_resources['observations'][:3], 1):  # Show first 3
        print(f"\nObservation {idx}:")
        print(json.dumps(obs, indent=2))
    if len(fhir_resources['observations']) > 3:
        print(f"\n... and {len(fhir_resources['observations']) - 3} more observations")

print("\n" + "=" * 80)
print("✅ All FHIR resources validated successfully!")
print("=" * 80)



FHIR RESOURCES (Pretty Print)

💊 CONDITIONS (1):

Condition 1:
{
  "resourceType": "Condition",
  "clinicalStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/condition-clinical",
        "code": "active"
      }
    ]
  },
  "verificationStatus": {
    "coding": [
      {
        "system": "http://terminology.hl7.org/CodeSystem/condition-ver-status",
        "code": "confirmed"
      }
    ]
  },
  "code": {
    "coding": [
      {
        "system": "http://hl7.org/fhir/sid/icd-10-cm",
        "code": "E66.3",
        "display": "Overweight"
      }
    ],
    "text": "Possible overweight"
  },
  "subject": {
    "reference": "Patient/patient-soap-01"
  },
  "recordedDate": "2025-11-30T18:10:48.354851Z"
}

💉 MEDICATIONS (1):

Medication 1:
{
  "resourceType": "MedicationRequest",
  "status": "active",
  "intent": "order",
  "medication": {
    "concept": {
      "coding": [
        {
          "system": "http://www.nlm.nih.gov/research/umls/rxn

In [93]:
# Test conversion on all SOAP notes we have results for
print("\n" + "=" * 80)
print("Testing FHIR conversion on multiple SOAP notes")
print("=" * 80)

fhir_results_all = {}

for soap_file, extracted_data in results.items():
    patient_id = f"patient-{soap_file.replace('.txt', '')}"
    fhir_result = convert_to_fhir(extracted_data, patient_id)
    fhir_results_all[soap_file] = fhir_result
    
    total = (
        (1 if fhir_result['patient'] else 0) +
        len(fhir_result['conditions']) +
        len(fhir_result['medications']) +
        len(fhir_result['observations'])
    )
    
    print(f"\n{soap_file}:")
    print(f"  • Total FHIR resources: {total}")
    print(f"    - Patient: {1 if fhir_result['patient'] else 0}")
    print(f"    - Conditions: {len(fhir_result['conditions'])}")
    print(f"    - Medications: {len(fhir_result['medications'])}")
    print(f"    - Observations: {len(fhir_result['observations'])}")

print("\n✅ FHIR conversion tested on all SOAP notes!")



Testing FHIR conversion on multiple SOAP notes

soap_01.txt:
  • Total FHIR resources: 13
    - Patient: 0
    - Conditions: 2
    - Medications: 1
    - Observations: 10

soap_02.txt:
  • Total FHIR resources: 10
    - Patient: 0
    - Conditions: 3
    - Medications: 1
    - Observations: 6

soap_03.txt:
  • Total FHIR resources: 0
    - Patient: 0
    - Conditions: 0
    - Medications: 0
    - Observations: 0

✅ FHIR conversion tested on all SOAP notes!
